#To DO
1. Run a model for each product
1. Recursive model 
1. Add Further Feature engineering - ambrosm https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model/

# Libraries

In [ ]:
pip install scikit-learn  -U

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

#import lightgbm as lgb 
#import xgboost as xgb
#from catboost import CatBoostRegressor

from sklearn.linear_model import LinearRegression,HuberRegressor,Ridge,TweedieRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import dateutil.easter as easter

import optuna
import math

In [ ]:
LINEAR_DATE_AUG = True

OPTUNA = False
NUM_TRIALS = 400


#Holidays
HOLIDAYS = True     
NEXT_HOLIDAY = False  

POST_PROCESSING = False
MODEL_TYPE = "Tweedie Regression"

VAL_SPLIT = "2017-12-31" #"2018-05-31"

In [ ]:
EPOCHS = 10000    
EARLY_STOPPING = 30
DEVICE = "cpu"

SCALER_NAME = "MinMax"  #None MinMax Standard
SCALER = MinMaxScaler()  #MinMaxScaler StandardScaler

obj is the objective function of the algorithm, i.e. what it's trying to maximize or minimize, e.g. "regression" means it's minimizing squared residuals.

Metric and eval are essentially the same. They are used for Early stopping 

# Load Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv",index_col = 0)
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",index_col = 0)
gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
gdp_df.set_index('year', inplace=True)

if HOLIDAYS:
    holidays = pd.read_csv("../input/holidays-finland-norway-sweden-20152019/Holidays_Finland_Norway_Sweden_2015-2019.csv",usecols = ["Date","Country","Name"]                      )
    holidays.rename(columns = {"Date":"date","Country":"country","Name":"holiday"},inplace= True)
    holidays["holiday"]= 1
    holidays["holiday"]= holidays["holiday"].astype("int32")
    holidays["date"] = pd.to_datetime(holidays["date"])

In [ ]:
#Make date
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

# Functions 

In [ ]:
def public_hols(df):
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df.fillna(value = 0,inplace=True)
    return df

In [ ]:
def get_gdp(row):
    country = 'GDP_' + row.country
    return gdp_df.loc[row.date.year, country]

In [ ]:
if HOLIDAYS:
    train = public_hols(train)
    test = public_hols(test)

Thanks to [ambrosm](https://www.kaggle.com/anirudhg15)

https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model/notebook#More-feature-engineering-(advanced-model)

In [ ]:
def engineer(df):
   
    #get GDP from file 
    df["gdp"] = df.apply(get_gdp, axis=1)   #improves Huber & Tweedie
    
    df["day"] = df["date"].dt.day
    df["dayofweek"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    
    #play around with if Tree model - each varies 
    #df['dayofyear'] = df['date'].dt.dayofyear                ### This can cause noise - remove but keep inverse
    df['inverse_dayofyear'] = 365 - df['date'].dt.dayofyear
    #df['quarter'] = 'Q' + df['date'].dt.quarter.astype(str)
    #df['daysinmonth'] = df['date'].dt.days_in_month           ## Also reduces performance
     
    if LINEAR_DATE_AUG:   
        for country in ['Finland', 'Norway']:
            df[country] = df.country == country
        df['KaggleRama'] = df.store == 'KaggleRama'
        for product in ['Kaggle Mug', 'Kaggle Sticker']:
            df[product] = df['product'] == product

        df["Friday"] = df["dayofweek"] ==4
        df["Sat_sun"] = (df["dayofweek"] ==5) |(df["dayofweek"] ==6)

        df.drop(["country","store","product"], axis =1, inplace = True)

        # Seasonal variations (Fourier series)
        dayofyear = df.date.dt.dayofyear
        for k in range(1, 21):
            df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
            df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
            df[f'mug_sin{k}'] = df[f'sin{k}'] * df['Kaggle Mug']
            df[f'mug_cos{k}'] = df[f'cos{k}'] * df['Kaggle Mug']
            df[f'sticker_sin{k}'] = df[f'sin{k}'] * df['Kaggle Sticker']
            df[f'sticker_cos{k}'] = df[f'cos{k}'] * df['Kaggle Sticker']

    return df

In [ ]:
def engineer_more(df):
    """Return a new dataframe with more engineered features"""
    new_df = engineer(df)

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)})],
                       axis=1)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) 
                                      for d in range(1, 13)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10)) + list(range(17, 25))})],
                       axis=1)
    
    # June
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) 
                                      for d in list(range(6, 14))})],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D"))
                                      for d in list(range(-5, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D"))
                                      for d in list(range(0, 10))})],
                       axis=1)
    
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})],
                       axis=1)
    
    # Growth is country-specific
    #for country in ['Finland', 'Norway', 'Sweden']:
    #    new_df[f"{country}_year"] = (df.country == country) * (df.date.dt.year - 2016)
    #    new_df[f"{country}_peak_year"] = (df.country == country) * (new_df.dec29 | new_df.dec30 | new_df.easter_week) * (df.date.dt.year - 2016)
        
    return new_df

train = engineer_more(train)
train['num_sold'] = train.num_sold.astype(np.float32)
test = engineer_more(test)


######################### CHECK THIS  -- WEIRD 
#test.year = 2018 # no growth patch, see https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/298318

features = test.columns
print(list(features))

In [ ]:
#train = engineer(train)
#test = engineer(test)

#Enconding the last features 
categorical_feats = [
    #country","store","product",
                     #"quarter",
                     #Friday",
                     #Sat_Sun"
                    ]

# Linear models improves with dropping first - Trees dont 
train = pd.get_dummies(train,columns= categorical_feats,drop_first=True)
test = pd.get_dummies(test,columns= categorical_feats,drop_first=True)

In [ ]:
def next_holiday(x):
    i=1
    while sum(holidays["date"] == pd.Timestamp(x) + pd.DateOffset(days=i)) ==0:
        i+=1
        if i >200:
            i=0
            break
            break
    return i

if NEXT_HOLIDAY:
    holidays["date"] = pd.to_datetime(holidays["date"])
    train["to_holiday"] = train["date"].apply(lambda x : next_holiday(x))
    test["to_holiday"] = test["date"].apply(lambda x : next_holiday(x))

In [ ]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [ ]:
def scale_data(X_train, test, X_test= None):
     
    scaler= SCALER
    
    #this can be train or X_train 
    X_train_s = scaler.fit_transform(X_train)
    test_s = scaler.transform(test)
    
   
    if X_test is not None:
        X_test_s = scaler.transform(X_test)
        return X_train_s, test_s , X_test_s
    
    else:
        return X_train_s, test_s 

In [ ]:
def create_lag(DAYS,df):
    df[f"shift{DAYS}"] = df.groupby(["store","product","country"])["num_sold"].shift(DAYS,fill_value = 0)
    return df

In [ ]:
def rolling_mean_std(day_roll_list, df):
    shift_days = day_roll_list[0]
    roll_window = day_roll_list[1]
    col_name = 'rolling_'+str(shift_days)+'_'+str(roll_window)
    df[col_name+"_mean"] = df.groupby(["store","product","country"])["num_sold"].shift(shift_days).rolling(roll_window).mean()
    df[col_name+"_std"] = df.groupby(["store","product","country"])["num_sold"].shift(shift_days).rolling(roll_window).std()
    
    return df

In [ ]:
prior_2017 = train[train["date"]<="2017-12-31"].index
after_2017 = train[train["date"]>"2017-12-31"].index

In [ ]:
train.index = train["date"]
train.drop("date",axis=1,inplace=True)

test.index = test["date"]
test.drop("date",axis=1,inplace=True)

# Split and Scale

In [ ]:
train = train.astype("float64")
test = test.astype("float64")

In [ ]:
X = train.drop("num_sold", axis=1)
y= train["num_sold"]

In [ ]:
X_train = train.iloc[prior_2017,:].drop("num_sold", axis=1)
X_test = train.iloc[after_2017,:].drop("num_sold", axis=1)
y_train= train.iloc[prior_2017,:]["num_sold"]
y_test= train.iloc[after_2017,:]["num_sold"]

# Run model

In [ ]:
params = {"power":1.0,
    "alpha":0.0,
    "fit_intercept":False,
    "link":'log', #‘auto’, ‘identity’, ‘log’}
    "tol":0.00000000001,
    "warm_start":False
         }

In [ ]:
def fit_model(X, y , test, X_test = None,y_test= None):
    
    model = TweedieRegressor(max_iter=EPOCHS, **params)
    
    # validation 
    if X_test is not None: 
        X_train_s, test_s , X_test_s = scale_data(X, test, X_test)
        
        model.fit(X_train_s,y)
        preds = model.predict(X_test_s)
        
        smape = SMAPE(y_test,preds)
        print("SMAPE:",smape )
        
        return preds, model, smape
    
    # Full prediction 
    else:
        X_s, test_s = scale_data(X, test)
        
        model.fit(X_s,y)
        preds = model.predict(test_s)
        
        return preds, model

In [ ]:
'''def fit_model(X_train,y_train,X_test = None,y_test= None):
    
    model = TweedieRegressor(max_iter=EPOCHS, **params)
    
    if X_test is None:
        model.fit(X_train,y_train)
        preds = model.predict(test)
        
    else:
        model.fit(X_train,y_train)
        preds = model.predict(X_test)
        print("SMAPE:", SMAPE(y_test,preds))

        
    return preds, model'''

In [ ]:
val_predictions, model, smape = fit_model(X_train,y_train,test, X_test,y_test)

In [ ]:
print("SMAPE :",SMAPE(y_test,val_predictions) )
print(f"\n EPOCHS: {EPOCHS}")
print(f"\n SCALER: {SCALER_NAME}")
print(f"\n POST_PROCESSING: {POST_PROCESSING}")

# Fit on Full Train data 

In [ ]:
test_predictions, model = fit_model(train.drop(["num_sold"],axis =1),train["num_sold"] , test)

In [ ]:
test_predictions

# Post Processing 

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv",index_col = 0)

In [ ]:
final_predictions = test_predictions.round()

In [ ]:
if POST_PROCESSING:
    # from previous run we are under predicting, lets scale the values upwards
    print("Scaling predictions ")
    print("preds_prior:", final_predictions)
    
    sub["num_sold"] = final_predictions*1.043
    
    print("preds after:", np.array(sub["num_sold"]))
else:
    sub["num_sold"] = final_predictions

# Submission 

In [ ]:
sub.to_csv("submission.csv")

In [ ]:
sub.head()

# Training Visualization

In [ ]:
sub.head()

In [ ]:
#for visual only
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",index_col = 0)
test["date"] = pd.to_datetime(test["date"])

fig,ax = plt.subplots(2,1, figsize=(25,20),sharey= True)

diff = y_test - val_predictions
sns.lineplot(ax=ax[0], data= y_test, label="Train Actual",ci=None)
sns.lineplot(ax=ax[0], data = y_test,x = y_test.index , y = val_predictions, label ="Validation Prediction" ,ci=None)
sns.lineplot(ax=ax[0],data =sub, x= test["date"], y = "num_sold",label="Final Prediction" ,ci=None) 

ax[0].set_title(f"Actual and Predicted Sales for {MODEL_TYPE}")

sns.lineplot(ax=ax[1], data = diff, label ="Residuals" )
ax[1].set_title(f"Validation Residuals")

plt.show()

In [ ]:
plt.figure(figsize=(25,10))

sns.lineplot(data= train["num_sold"] ,label="Train Actual",ci=None)
sns.lineplot(data =sub, x= test["date"], y = "num_sold",label="Final Prediction" ,ci=None) 
plt.title("Actual and Predicted Sales")

plt.show()